# Plonk Course - Lecture 4 - 算术约束与拷贝约束

## 电路

对于下图这样的一个电路，Prover 取定隐私数据 `witness = [x1, x2, x3, x4]` 后，想对 Verifier 证明这些 `witness` 经过这样的电路运算后的值为 `out`。

![](./img/circuit.png)

在下面的运算中取 `witness = [x1, x2, x3, x4] = [1, 2, 3, 11]`，`out = 99`。我们也可以自己验证下这组值是否满足上述电路。上述电路满足下面三个约束：

- $x_1 + x_2 = x_6$
- $x_3 \cdot x_4 = x_5$
- $x_6 \cdot x_5 = out$

那么代入 `witness = [x1, x2, x3, x4] = [1, 2, 3, 11]` 可得

- $x_6 = x_1 + x_2 = 1 + 2 = 3$
- $x_5 = x_3 \cdot x_4 = 3 \cdot 11 = 33$
- $out = x_6 \cdot x_5 = 3 \cdot 33 = 99$

因此 Prover 诚实地运算了上述电路，电路输出的 `out = 99` 是正确的。如果 Prover 想要在区块链上向 Verifier 简洁地证明这件事情，并不向 Verifer 暴露 `witness`，就不能是我们这样直接带入 `witness` 进行计算了。我们用 Plonk 协议来实现这一点。

## 矩阵

Prover 和 Verifier 共识的 $Q$ 矩阵为

$$
\begin{array}{c|c|c|c|}
i & q_L & q_R & q_M & q_C & q_O  \\
\hline
0 & 0 & 0 & 0 & 99& 1 \\
1 & 0 & 0 & 1 & 0& 1 \\
2 & 1 & 1 & 0 & 0& 1 \\
3 & 0 & 0 & 1 & 0& 1 \\
\end{array}
$$

Prover 填写电路的 $W$ 矩阵 (对 Verifier 保密) 

$$
\begin{array}{c|c|c|c|}
i & w_{a,i} & w_{b,i} & w_{c,i}  \\
\hline
0 & 0 & 0 & {\color{green}out} \\
1 & {\color{red}x_6} & {\color{blue}x_5} & {\color{green}out} \\
2 & x_1 & x_2 & {\color{red}x_6} \\
3 & x_3 & x_4 & {\color{blue}x_5} \\
\end{array}
$$

表格中相同颜色的地方需要 Prover 填写相同的值，也就是拷贝约束，Prover 需要向 Verifier 证明自己正确的填写了相同的值。

代入我们上面取的值，Prover 如实填写的 $W$ 矩阵为

$$
\begin{array}{c|c|c|c|}
i & w_{a,i} & w_{b,i} & w_{c,i}  \\
\hline
0 & 0 & 0 & {\color{green}99} \\
1 & {\color{red}3} & {\color{blue}33} & {\color{green}99} \\
2 & 1 & 2 & {\color{red}3} \\
3 & 3 & 11 & {\color{blue}33} \\
\end{array}
$$

位置编码矩阵

$$
\begin{array}{c|c|c|c|}
i & id_{a,i} & id_{b,i} & id_{c,i}  \\
\hline
0 & 0 & 4 & {\color{green}8} \\
1 & {\color{red}1} & {\color{blue}5} & {\color{green}9} \\
2 & 2 & 6 & {\color{red}10} \\
3 & 3 & 7 & {\color{blue}11} \\
\end{array}
$$

对应置换矩阵

$$
\begin{array}{c|c|c|c|}
i & \sigma_{a,i} & \sigma_{b,i} & \sigma_{c,i}  \\
\hline
0 & 0 & 4 & {\color{green}9} \\
1 & {\color{red}10} & {\color{blue}11} & {\color{green}8} \\
2 & 2 & 6 & {\color{red}1} \\
3 & 3 & 7 & {\color{blue}5} \\
\end{array}
$$

需要对上述矩阵中的每一列进行多项式编码，同时也要选定我们编码的有限域。

### 有限域

在 $W$ 矩阵，其中数的范围为 0 ～ 99 ，则 $\mathbb{F}$ 的大小至少为 100，可选取有限域的大小为素数 $p = 101$，即 $\mathbb{F}_p = \mathbb{F}_{101}$，其生成元为 2 。

In [3]:
# 有限域 F_101
F_101 = GF(101)

# 获取 F_101 的生成元
g_F = F_101.multiplicative_generator()

print("F_101的生成元为：", g_F)

F_101的生成元为： 2


对 $p - 1$ 作因子分解 $p - 1 = 100 = 2^2 \cdot 5^2$，则 $\mathbb{F}_{101}$ 存在一个大小为 4 的乘法子群，其生成元 $\omega = 10$，$\mathbb{H} = \{\omega^0 = 1, \omega^1 = 10, \omega^2 = {10}^2 = 100, \omega^3 = 10^3 = 91\}$。

In [6]:
# 验证 omega = 10 是否为群 H 的生成元
omega = F_101(10)
n = 8
for i in range(0, n + 1):
    print("%d^%d: %d" %(omega, i, F_101(omega^i)), end = ", ")

10^0: 1, 10^1: 10, 10^2: 100, 10^3: 91, 10^4: 1, 10^5: 10, 10^6: 100, 10^7: 91, 10^8: 1, 

In [8]:
# H 中的元素 H = {omega^0, omega^1, omega^2, omega^3}
H = [F_101(omega^0), F_101(omega^1), F_101(omega^2), F_101(omega^3)]
print("H = ", H)

H =  [1, 10, 100, 91]


## 多项式编码

多项式编码即是对于一组值 $f = \{f_0, f_1, f_2, \cdots, f_{n - 1}\}$ 在乘法子群 $\mathbb{H} = \{\omega_0, \omega_1, \omega_2, \cdots, \omega_{n-1}\} = \{\omega, \omega^1, \omega^2, \cdots, \omega^{n-1}\}$ ( $\omega$ 为乘法群 $\mathbb{H}$ 的生成元) 上进行 Lagrange 插值。也就是得到的多项式 $f(X)$ 满足

$$
\begin{split}
f(\omega_0) & = f_0\\
f(\omega_1) & = f_1\\
f(\omega_2) & = f_2\\
& \cdots \\
f(\omega_{n-1}) & = f_{n-1}\\
\end{split}
$$

用 Lagarange 插值来计算 $f(X)$，即

$$
f(X) = \sum_{i = 0}^{n - 1} f_i \cdot L_i(X)
$$

其中

$$
L_i(x) = \prod_{i = 0, i \neq j}^3 \frac{x - \omega_j}{\omega_i - \omega_j}
$$

In [11]:
# Lagrange 插值计算
def lagrange_interpolation(GF, H, f):
    n = len(f)
    # 先计算 L_i(X) 的分母
    L_denominator = [1] * n

    for i in range(0, n):
        for j in range(0, n):
            if i != j:
                L_denominator[i] = GF((H[i] - H[j]) * L_denominator[i])

    # 声明多项式自变量 X 在 GF 中
    R.<X> = GF[]

    # 计算 L_i(X)
    L_X = [1] * n
    for i in range(0, n):
        for j in range(0, n):
            if i != j:
                L_X[i] = (X - H[j]) * L_X[i]
        L_X[i] = (GF(1) / L_denominator[i]) * L_X[i]
    
    # 计算 f(X)
    f_X = 0
    for i in range(0, n):
        f_X = f_X + f[i] * L_X[i]

    return f_X

## 预处理

Prover 与 Verifier 需要构造 $[q_L(X)]$， $[q_R(X)]$， $[q_O(X)]$， $[q_M(X)]$， $[q_C(X)]$， $[{\sigma_a}(X)]$， $[{\sigma_b}(X)]$， $[{\sigma_c}(X)]$。
这里要用到多项式来进行编码。

### 对 $Q$ 进行多项式编码
$Q$ 矩阵为

$$
\begin{array}{c|c|c|c|}
i & q_L & q_R & q_M & q_C & q_O  \\
\hline
0 & 0 & 0 & 0 & 99& 1 \\
1 & 0 & 0 & 1 & 0& 1 \\
2 & 1 & 1 & 0 & 0& 1 \\
3 & 0 & 0 & 1 & 0& 1 \\
\end{array}
$$

In [15]:
# 计算 q_L(X), q_R(X), q_M(X), q_C(X), q_O(X)
q_L = [0, 0, 1, 0]
q_R = [0, 0, 1, 0]
q_M = [0, 1, 0, 1]
q_C = [99, 0, 0, 0]
q_O = [1, 1, 1, 1]

q_L_X = lagrange_interpolation(F_101, H, q_L)
q_R_X = lagrange_interpolation(F_101, H, q_R)
q_M_X = lagrange_interpolation(F_101, H, q_M)
q_C_X = lagrange_interpolation(F_101, H, q_C)
q_O_X = lagrange_interpolation(F_101, H, q_O)

print("q_L(X) = ", q_L_X)
print("q_R(X) = ", q_R_X)
print("q_M(X) = ", q_M_X)
print("q_C(X) = ", q_C_X)
print("q_O(X) = ", q_O_X)

q_L(X) =  25*X^3 + 76*X^2 + 25*X + 76
q_R(X) =  25*X^3 + 76*X^2 + 25*X + 76
q_M(X) =  50*X^2 + 51
q_C(X) =  50*X^3 + 50*X^2 + 50*X + 50
q_O(X) =  1


### 位置向量的优化

#### 方案一：自然数

用自然数的方式来编号是比较自然的，即 $(0, 1, 2, \cdots, 11)$。

位置编码矩阵

$$
\begin{array}{c|c|c|c|}
i & id_{a,i} & id_{b,i} & id_{c,i}  \\
\hline
0 & 0 & 4 & {\color{green}8} \\
1 & {\color{red}1} & {\color{blue}5} & {\color{green}9} \\
2 & 2 & 6 & {\color{red}10} \\
3 & 3 & 7 & {\color{blue}11} \\
\end{array}
$$

对应置换矩阵

$$
\begin{array}{c|c|c|c|}
i & \sigma_{a,i} & \sigma_{b,i} & \sigma_{c,i}  \\
\hline
0 & 0 & 4 & {\color{green}9} \\
1 & {\color{red}10} & {\color{blue}11} & {\color{green}8} \\
2 & 2 & 6 & {\color{red}1} \\
3 & 3 & 7 & {\color{blue}5} \\
\end{array}
$$

In [18]:
# 方案一：计算 sigma_a(X), sigma_b(X), sigma_c(X)
sigma_a_1 = [0, 10, 2, 3]
sigma_b_1 = [4, 11, 6, 7]
sigma_c_1 = [9, 8, 1, 5]

sigma_a_1_X = lagrange_interpolation(F_101, H, sigma_a_1)
sigma_b_1_X = lagrange_interpolation(F_101, H, sigma_b_1)
sigma_c_1_X = lagrange_interpolation(F_101, H, sigma_c_1)

print("sigma_a_1(X) = ", sigma_a_1_X)
print("sigma_b_1(X) = ", sigma_b_1_X)
print("sigma_c_1(X) = ", sigma_c_1_X)

sigma_a_1(X) =  17*X^3 + 73*X^2 + 83*X + 29
sigma_b_1(X) =  60*X^3 + 99*X^2 + 40*X + 7
sigma_c_1(X) =  60*X^3 + 75*X^2 + 45*X + 31


#### 方案二：乘法子群 $\mathbb{H}$ 的陪集

在编码 $[{\sigma_a}(X)]$， $[{\sigma_b}(X)]$， $[{\sigma_c}(X)]$ 多项式时，不一定要采用自然数编号，满足互不相等的值来标记置换即可。为了使 Verifier 自己能高效计算，可以用乘法子群不同的陪集中的元素，即

$$
\begin{split}
\vec{id}_a &= (1,\omega,\omega^2,\omega^3)\\
\vec{id}_b &= (k_1,k_1\omega,k_1\omega^2,k_1\omega^3)\\
\vec{id}_c &= (k_2,k_2\omega,k_2\omega^2,k_2\omega^3)\\
\end{split}
$$

其中 $k_i$ 的取法要保证得到的三个不同的陪集，即得到的三个陪集 $\mathbb{H}$， $k_1\mathbb{H}$ 与 $k_2 \mathbb{H}$ 不能有交集，如果有交集，那么这种编号方式就有重复的两个元素了，从而无法达到我们想要用来进行置换证明的目的。这样取的目的，让我们可以直接得到 ${id_a}(X)$， ${id_a}(X)$ 与 ${id_a}(X)$ 的表达式，无需再用 Lagrange 插值进行计算，这对 Verifier 是非常友好的。

$$
{id_a}(X) = X, \quad {id_b}(X) = k_1\cdot X, \quad  {id_c}(X) = k_2\cdot X
$$

In [21]:
# 生成 k_1 和 k_2
k_1 = F_101.random_element()
k_2 = F_101.random_element()

# k_1 不能为 0， 否则 k_1H 中每个元素为0；且不能为 H 中的元素，否则生成的陪集会和 H 相同
for i in range(0, 4):
    while k_1 == 0 or k_1== H[i]:
        k_1 = F_101.random_element()
        
#  k_2 不能为 0， 否则 k_2H 中每个元素为0；且不能是陪集 H 与 k_1H 中的元素，否则生成的陪集 k_2H 会和 H 或者 k_1H 相等
for i in range(0, 4):
    k_1H = [k_1 * H[0], k_1 * H[1], k_1 * H[2], k_1 * H[3]]
    while k_2 == 0 or k_2 == H[i] or k_2 == k_1H[i]:
        k_2 = F_101.random_element()

print("k_1 = ", k_1, ", k_2 = ", k_2)

# 计算陪集 H, k_1H, k_2H 中的元素
k_1H = [k_1 * H[0], k_1 * H[1], k_1 * H[2], k_1 * H[3]]
k_2H = [k_2 * H[0], k_2 * H[1], k_2 * H[2], k_2 * H[3]]
print("H = ", H)
print("k_1H = ", k_1H)
print("k_2H = ", k_2H)

k_1 =  6 , k_2 =  33
H =  [1, 10, 100, 91]
k_1H =  [6, 60, 95, 41]
k_2H =  [33, 27, 68, 74]


##### Bad Case

$k_1$ 与 $k_2$ 不能取自同一个陪集中的元素。

In [24]:
# bad case
bad_k_1 = F_101.random_element()
bad_k_1H = [bad_k_1 * H[0], bad_k_1 * H[1], bad_k_1 * H[2], bad_k_1 * H[3]]
print("H = ", H)
print("bad_k_1H = ", bad_k_1H)

H =  [1, 10, 100, 91]
bad_k_1H =  [15, 49, 86, 52]


现在故意取 bad_k2 为 bad_k_1H 中的元素

In [27]:
bad_k2 = bad_k_1H[2]
bad_k_2H = [bad_k2 * H[0], bad_k2 * H[1], bad_k2 * H[2], bad_k2 * H[3]]
print("bad_k_2H = ", bad_k_2H)

bad_k_2H =  [86, 52, 15, 49]


可以看到 bad_k_2H 与 bad_k_1H 中的元素完全相同。

位置编码矩阵

$$
\begin{array}{c|c|c|c|}
i & id_{a,i} & id_{b,i} & id_{c,i}  \\
\hline
0 & 1 & k_1 & {\color{green}k_2} \\
1 & {\color{red}\omega} & {\color{blue}k_1\omega} & {\color{green}k_2\omega} \\
2 & \omega^2 & k_1\omega^2 & {\color{red}k_2\omega^2} \\
3 & \omega^3 & k_1\omega^3 & {\color{blue}k_2\omega^3} \\
\end{array}
$$

对应置换矩阵

$$
\begin{array}{c|c|c|c|}
i & \sigma_{a,i} & \sigma_{b,i} & \sigma_{c,i}  \\
\hline
0 & 1 & k_1 & {\color{green}k_2\omega} \\
1 & {\color{red}k_2\omega^2} & {\color{blue}k_2\omega^3} & {\color{green}k_2} \\
2 & \omega^2 & k_1\omega^2 & {\color{red}\omega} \\
3 & \omega^3 & k_1\omega^3 & {\color{blue}k_1\omega} \\
\end{array}
$$

In [31]:
# 计算 sigma_a(X), sigma_b(X), sigma_c(X)
sigma_a = [1, k_2 * omega^2, omega^2, omega^3]
sigma_b = [k_1, k_2 * omega^3, k_1 * omega^2, k_1 * omega^3]
sigma_c = [k_2 * omega, k_2, omega, k_1 * omega]

print("sigma_a = ", sigma_a)
print("sigma_b = ", sigma_b)
print("sigma_c = ", sigma_c)

sigma_a_X = lagrange_interpolation(F_101, H, sigma_a)
sigma_b_X = lagrange_interpolation(F_101, H, sigma_b)
sigma_c_X = lagrange_interpolation(F_101, H, sigma_c)

print("sigma_a(X) = ", sigma_a_X)
print("sigma_b(X) = ", sigma_b_X)
print("sigma_c(X) = ", sigma_c_X)

sigma_a =  [1, 68, 100, 91]
sigma_b =  [6, 74, 95, 41]
sigma_c =  [27, 33, 10, 60]
sigma_a(X) =  44*X^3 + 36*X^2 + 58*X + 65
sigma_b(X) =  35*X^3 + 47*X^2 + 72*X + 54
sigma_c(X) =  63*X^3 + 87*X^2 + 97*X + 83


## 第一步

Prover 针对 $W$ 表格的每一列，构造 $[w_a(X)]$， $[w_b(X)]$， $[w_c(X)]$， $\phi(X)$ 使得

$$
q_L(X)w_a(X)+q_R(X)w_b(X)+ q_M(X)w_a(X)w_b(X) - q_O(X)w_c(X)+q_C(X) + \phi(X) = 0
$$

$W$ 矩阵为

$$
\begin{array}{c|c|c|c|}
i & w_{a,i} & w_{b,i} & w_{c,i}  \\
\hline
0 & 0 & 0 & {\color{green}99} \\
1 & {\color{red}3} & {\color{blue}33} & {\color{green}99} \\
2 & 1 & 2 & {\color{red}3} \\
3 & 3 & 11 & {\color{blue}33} \\
\end{array}
$$

In [35]:
# 计算 w_a(X), w_b(X), w_c(X)
w_a = [0, 3, 1, 3]
w_b = [0, 33, 2, 11]
w_c = [99, 99, 3, 33]

w_a_X = lagrange_interpolation(F_101, H, w_a)
w_b_X = lagrange_interpolation(F_101, H, w_b)
w_c_X = lagrange_interpolation(F_101, H, w_c)

print("w_a(X) = ", w_a_X)
print("w_b(X) = ", w_b_X)
print("w_c(X) = ", w_c_X)

# phi(X)
R.<X> = F_101[]
phi_X = 0 * X

print("phi(X) = ", phi_X)

w_a(X) =  25*X^3 + 24*X^2 + 25*X + 27
w_b(X) =  4*X^3 + 40*X^2 + 96*X + 62
w_c(X) =  88*X^3 + 43*X^2 + 61*X + 8
phi(X) =  0


## 第二步

Verifier 发送随机数 $\beta$ 与 $\gamma$。

In [38]:
beta = F_101.random_element()
gamma = F_101.random_element()
print("beta = ", beta, ", gamma = ", gamma)

beta =  91 , gamma =  73


## 第三步

Prover 构造 $[z(X)]$，使得

$$
\begin{split}
L_0(X)(z(X)-1) &= 0 \\
z(\omega\cdot X)g(X) -  z(X)f(X) &=0
\end{split}
$$

### 位置编码矩阵的选择
#### 方案一：自然数

$$
\begin{array}{c|c|c|c|}
i & id_{a,i} & id_{b,i} & id_{c,i}  \\
\hline
0 & 0 & 4 & {\color{green}8} \\
1 & {\color{red}1} & {\color{blue}5} & {\color{green}9} \\
2 & 2 & 6 & {\color{red}10} \\
3 & 3 & 7 & {\color{blue}11} \\
\end{array}
$$

In [42]:
# 方案一：自然数生成的 id_1
id_a_1 = [0, 1, 2, 3]
id_b_1 = [4, 5, 6, 7]
id_c_1 = [8, 9, 10, 11]

#### 方案二：乘法子群 $\mathbb{H}$ 的陪集
$$
\begin{array}{c|c|c|c|}
i & id_{a,i} & id_{b,i} & id_{c,i}  \\
\hline
0 & 1 & k_1 & {\color{green}k_2} \\
1 & {\color{red}\omega} & {\color{blue}k_1\omega} & {\color{green}k_2\omega} \\
2 & \omega^2 & k_1\omega^2 & {\color{red}k_2\omega^2} \\
3 & \omega^3 & k_1\omega^3 & {\color{blue}k_2\omega^3} \\
\end{array}
$$

In [45]:
# 方案二：陪集生成的 id_2
id_a = [F_101(1), omega, omega^2, omega^3]
id_b = [k_1, k_1 * omega, k_1 * omega^2, k_1 * omega^3]
id_c = [k_2, k_2 * omega, k_2 * omega^2, k_2 * omega^3]

print("id_a = ", id_a)
print("id_b = ", id_b)
print("id_c = ", id_c)

# Lagrange 插值计算 id_a(X), id_b(X), id_c(X)
# 与理论结果一致, id_a(X) = X, id_b(X) = k_1 * X, id_c(X) = k_2 * X
# 实际计算时直接代上述方程
id_a_X = lagrange_interpolation(F_101, H, id_a)
id_b_X = lagrange_interpolation(F_101, H, id_b)
id_c_X = lagrange_interpolation(F_101, H, id_c)

print("id_a(X) = ", id_a_X)
print("id_b(X) = ", id_b_X)
print("id_c(X) = ", id_c_X)

id_a =  [1, 10, 100, 91]
id_b =  [6, 60, 95, 41]
id_c =  [33, 27, 68, 74]
id_a(X) =  X
id_b(X) =  6*X
id_c(X) =  33*X


下面的计算以方案二的计算为例，将下述代码中的 `sigma_a, sigma_b, sigma_c, id_a, id_b, id_c` 分别替换为`sigma_a_1, sigma_b_1, sigma_c_1, id_a_1, id_b_1, id_c_1` 即可计算对应方案一中的多项式了。

### 计算 $f(X)$ 与 $g(X)$

$f(X)$ 与 $g(X)$ 中的值分别为 $\{f_i\}$ 与 $\{g_i\}$ 为

$$
\begin{split}
f_i &= (w_{a,i}+\beta\cdot id_{a,i}+\gamma)(w_{b,i}+\beta\cdot id_{b,i}+\gamma)(w_{c,i}+\beta\cdot id_{c,i}+\gamma) \\
g_i &= (w'_{a,i}+\beta\cdot \sigma_{a,i}+\gamma)(w'_{b,i}+\beta\cdot \sigma_{b,i}+\gamma)(w'_{c,i}+\beta\cdot \sigma_{c,i}+\gamma)
\end{split}
$$

在 $g_i$ 中 $w'_{a,i} = w_{a,i}$ ，$w'_{b,i} = w_{b,i}$ ，$w'_{c,i} = w_{c,i}$，因此

$$
\begin{split}
f_i &= (w_{a,i}+\beta\cdot id_{a,i}+\gamma)(w_{b,i}+\beta\cdot id_{b,i}+\gamma)(w_{c,i}+\beta\cdot id_{c,i}+\gamma) \\
g_i &= (w_{a,i}+\beta\cdot \sigma_{a,i}+\gamma)(w_{b,i}+\beta\cdot \sigma_{b,i}+\gamma)(w_{c,i}+\beta\cdot \sigma_{c,i}+\gamma)
\end{split}
$$

那么多项式 $f(X)$ 与 $g(X)$ 满足
$$
\begin{split}
f(X)&=\Big(w_a(X)+\beta\cdot {id_a}(X)+\gamma\Big)\Big(w_b(X)+\beta\cdot {id_b}(X)+\gamma\Big)\Big(w_c(X)+\beta\cdot {id_c}(X)+\gamma\Big)\\
g(X)&=\Big(w_a(X)+\beta\cdot {\sigma_a}(X)+\gamma\Big)\Big(w_b(X)+\beta\cdot {\sigma_b}(X)+\gamma\Big)\Big(w_c(X)+\beta\cdot {\sigma_c}(X)+\gamma\Big)\\
\end{split}
$$

In [49]:
# 计算 f(X) 与 g(X)
f_X = (w_a_X + beta * X + gamma) * (w_b_X + beta * k_1 * X + gamma) * (w_c_X + beta * k_2 * X + gamma)
g_X = (w_a_X + beta * sigma_a_X + gamma) * (w_b_X + beta * sigma_b_X + gamma) * (w_c_X + beta * sigma_c_X + gamma)

print("f(X) = ", f_X)
print("g(X) = ", g_X)

f(X) =  13*X^9 + 51*X^8 + 15*X^7 + 25*X^6 + 39*X^5 + 70*X^4 + 77*X^3 + 50*X^2 + 70*X + 74
g(X) =  73*X^9 + 42*X^8 + 79*X^7 + 47*X^6 + 21*X^5 + 15*X^4 + 91*X^3 + 2*X^2 + 38*X + 29


In [51]:
# 计算 f 与 g
f = [f_X(omega^0), f_X(omega^1), f_X(omega^2), f_X(omega^3)]
g = [g_X(omega^0), g_X(omega^1), g_X(omega^2), f_X(omega^3)]

print("f = ", f, " g = ", g)

f =  [80, 16, 56, 22]  g =  [33, 61, 35, 22]


### 计算 $z(X)$

上面计算出 $f_i$ 与 $g_i$ ，那么由递推公式

$$
z_0 = 1, \qquad z_{i+1}=z_i\cdot \frac{f_i}{g_i}\\
$$

可计算出 $z_i$。

$$
\begin{array}{|c|c|c|}
i & \omega_i \in \mathbb{H} & z_i\\
\hline
0 & \omega_0 = \omega^0 =1 & 1\\ 
1 & \omega_1 = \omega^1 =10 & 1\cdot \frac{f_0}{g_0} \\ 
2 & \omega_2 = \omega^2 =100 & \frac{f_0}{g_0}\cdot \frac{f_1}{g_1}\\
3 & \omega_3 = \omega^3 =91 & \frac{f_0f_1}{g_0g_1}\cdot \frac{f_2}{g_2} \\
4 & \omega_4 = \omega^4 =1 & \frac{f_0f_1f_2f_3}{g_0g_1g_2g_3}=1\\
\end{array}
$$

In [54]:
# 计算 z_i
z = [1, 0, 0, 0]     # 初始化
for i in range(1, 4):
    z[i] = F_101(z[i - 1] * (f[i - 1] / g[i - 1]))
print("z_i = ", z)

# 计算 z(X)
z_X = lagrange_interpolation(F_101, H, z)

print("z(X) = ", z_X)

z_i =  [1, 82, 48, 97]
z(X) =  77*X^3 + 18*X^2 + 51*X + 57


如果我们要计算出 $z(\omega \cdot X)$ 的多项式，则依然可以用 Lagrange 插值

$$
\begin{array}{|c|c|c|}
i & \omega_i \in \mathbb{H} & z_{\omega,i}\\
\hline
0 & \omega_0 = \omega^0 =1 & z(\omega)\\ 
1 & \omega_1 = \omega^1 =10 & z(\omega^2) \\ 
2 & \omega_2 = \omega^2 =100 & z(\omega^3) \\
3 & \omega_3 = \omega^3 =91 & z(\omega^4) \\
\end{array}
$$

In [57]:
# 计算 z(omega * X)
z_omega = [z_X(omega), z_X(omega^2), z_X(omega^3), z_X(omega^4)]
print("z_omega = ", z_omega)

z_omega_X = lagrange_interpolation(F_101, H, z_omega)
print("z_omega_X = ", z_omega_X)

z_omega =  [82, 48, 97, 1]
z_omega_X =  38*X^3 + 83*X^2 + 5*X + 57


## 第四步

Verifier 发送随机挑战数 $\alpha$。

In [60]:
alpha = F_101.random_element()
print("alpha = ", alpha)

alpha =  31


## 第五步

Prover 计算 $h(X)$，并构造商多项式 $[t(X)]$

$$
\begin{split}
h(X) = &\ q_L(X)w_a(X)+q_R(X)w_b(X)+ q_M(X)w_a(X)w_b(X) - q_O(X)w_c(X)+q_C(X) + \phi(X) \\
 & + \alpha(z(\omega X)\cdot g(X)-z(X)\cdot f(X)) + \alpha^2(L_0(X)\cdot(z(X)-1))
\end{split}
$$

其中

$$
\begin{split}
f(X)&=\Big(w_a(X)+\beta\cdot {id_a}(X)+\gamma\Big)\Big(w_b(X)+\beta\cdot {id_b}(X)+\gamma\Big)\Big(w_c(X)+\beta\cdot {id_c}(X)+\gamma\Big)\\
g(X)&=\Big(w_a(X)+\beta\cdot {\sigma_a}(X)+\gamma\Big)\Big(w_b(X)+\beta\cdot {\sigma_b}(X)+\gamma\Big)\Big(w_c(X)+\beta\cdot {\sigma_c}(X)+\gamma\Big)\\
\end{split}
$$

其中商多项式 $t(X)=\frac{h(X)}{z_H(X)}$。


### 计算 $h(X)$

In [64]:
# 计算 L_0(X)
L0 = [1, 0, 0, 0]
L0_X = lagrange_interpolation(F_101, H, L0)
print("L0(X) = ", L0_X)

# 计算 h(X)
h_X = q_L_X * w_a_X + q_R_X * w_b_X + q_M_X * w_a_X * w_b_X - q_O_X * w_c_X + q_C_X + phi_X + alpha * (z_omega_X * g_X - z_X * f_X) + alpha * alpha * (L0_X * (z_X - 1))

print("h(X) = ", h_X)

L0(X) =  76*X^3 + 76*X^2 + 76*X + 76
h(X) =  19*X^12 + 42*X^11 + 64*X^10 + 85*X^9 + 93*X^8 + 16*X^7 + 6*X^6 + 61*X^5 + 29*X^4 + 43*X^3 + 31*X^2 + 56*X + 61


### 计算 $t(X)$

因此得到的商多项式 $t(X)$ 为

$$
t(X) = \frac{h(X)}{z_H(X)}
$$

其中 $z_H(X)$ 称为 vanishing polynomial ，即

$$
z_H(X) = (X - \omega^0)(X - \omega^1)(X - \omega^2)(X - \omega^3) = X^4 - 1
$$

In [68]:
# 计算 z_H(X)
z_H_X = X^4 - 1
print("z_H(X) = ", z_H_X)

# 计算 t(X)
t_X = h_X / z_H_X

print("t(X) = ", t_X)

quotient, remainder = h_X.quo_rem(z_H_X)
print(f"Quotient: {quotient}")
print(f"Remainder: {remainder}") 

# 确保计算正确，z_H(X) 能够整除 t(X)
assert remainder == 0

z_H(X) =  X^4 + 100
t(X) =  19*X^8 + 42*X^7 + 64*X^6 + 85*X^5 + 11*X^4 + 58*X^3 + 70*X^2 + 45*X + 40
Quotient: 19*X^8 + 42*X^7 + 64*X^6 + 85*X^5 + 11*X^4 + 58*X^3 + 70*X^2 + 45*X + 40
Remainder: 0


## 第六步
Verifier 发送随机挑战数 $\zeta$，查询上述的所有 Oracle，得到
- $\bar{w}_a=w_a(\zeta)$， $\bar{w}_b=w_b(\zeta)$， $\bar{w}_c=w_c(\zeta)$
- $\bar{q}_L=q_L(\zeta)$， $\bar{q}_R=q_R(\zeta)$， $\bar{q}_M=q_M(\zeta)$，  $\bar{q}_O=q_O(\zeta)$， $\bar{q}_C=q_C(\zeta)$
- $\bar{\sigma}_a=\sigma_a(\zeta)$， $\bar{\sigma}_b=\sigma_b(\zeta)$， $\bar{\sigma}_c=\sigma_c(\zeta)$
- $\bar{z}_{(\omega\cdot\zeta)}=z(\omega\cdot\zeta)$， $\bar{z}_{(\zeta)}=z(\zeta)$
- $\bar{t}=t(\zeta)$

In [71]:
# 随机数 zeta
zeta = F_101.random_element()
print("zeta = ", zeta)

zeta =  16


### Oracle 查询

In [74]:
# 查询 w_a(X), w_b(X), w_c(X)

w_a_bar = w_a_X(zeta)
w_b_bar = w_b_X(zeta)
w_c_bar = w_c_X(zeta)
print("w_a_bar = ", w_a_bar, ", w_b_bar = ", w_b_bar, ", w_c_bar = ", w_c_bar)

w_a_bar =  93 , w_b_bar =  43 , w_c_bar =  53


In [76]:
# 查询 q_L(X), q_R(X), q_M(X), q_O(X), q_C(X)

q_L_bar = q_L_X(zeta)
q_R_bar = q_R_X(zeta)
q_M_bar = q_M_X(zeta)
q_O_bar = q_O_X(zeta)
q_C_bar = q_C_X(zeta)

print("q_L_bar = ", q_L_bar, ", q_R_bar = ", q_R_bar, "q_M_bar = ", q_M_bar, ", q_O_bar = ", q_O_bar, ", q_C_bar = ", q_C_bar)

q_L_bar =  21 , q_R_bar =  21 q_M_bar =  24 , q_O_bar =  1 , q_C_bar =  88


In [78]:
# 查询 sigma_a(X), sigma_b(X), sigma_c(X)

sigma_a_bar = sigma_a_X(zeta)
sigma_b_bar = sigma_b_X(zeta)
sigma_c_bar = sigma_c_X(zeta)

print("sigma_a_bar = ", sigma_a_bar, ", sigma_b_bar = ", sigma_b_bar, ", sigma_c_bar = ", sigma_c_bar)

sigma_a_bar =  48 , sigma_b_bar =  48 , sigma_c_bar =  64


In [80]:
# 查询 z(X), z(omega * X), t(X)

z_omega_bar = z_omega_X(zeta)
z_bar = z_X(zeta)

print("z_omega_bar = ", z_omega_bar, ", z_bar = ", z_bar)

t_bar = t_X(zeta)

print("t_bar = ", t_bar)

z_omega_bar =  81 , z_bar =  97
t_bar =  88


### Verifier 计算
Verifier 还要自行计算
- $\bar{f}_{(\zeta)} =(\bar{w}_a+\beta\cdot \zeta + \gamma) (\bar{w}_b+\beta\cdot k_1\cdot \zeta +\gamma)(\bar{w}_c+\beta\cdot k_2 \cdot \zeta +\gamma)$
- $\bar{g}_{(\zeta)}=(\bar{w}_a+\beta\cdot \bar{\sigma}_1 + \gamma) (\bar{w}_b+\beta\cdot\bar{\sigma}_2+\gamma)(\bar{w}_c+\beta\cdot\bar{\sigma}_3+\gamma)$
- $L_0(\zeta)$
- $z_H(\zeta)$
- $\phi(\zeta)$

In [83]:
# Verifier 计算 f(zeta), g(zeta)
f_v_bar = (w_a_bar + beta * zeta + gamma) * (w_b_bar + beta * k_1 * zeta + gamma) * (w_c_bar + beta * k_2 * zeta + gamma)
g_v_bar = (w_a_bar + beta * sigma_a_bar + gamma) * (w_b_bar + beta * sigma_b_bar + gamma) * (w_c_bar + beta * sigma_c_bar + gamma)

print("f_v_bar = ", f_v_bar, ", g_v_bar = ", g_v_bar)

f_v_bar =  42 , g_v_bar =  21


In [85]:
# Verifier 计算 L_0(zeta)
L0_v = [1, 0, 0, 0]
L0_v_X = lagrange_interpolation(F_101, H, L0_v)
L0_v_zeta = L0_v_X(zeta)
print("L0_v_zeta = ", L0_v_zeta)

# Verifier 计算 z_H(zeta)
z_H_v_X = X^4 - 1
z_H_v_zeta = z_H_v_X(zeta)
print("z_H_v_zeta = ", z_H_v_zeta)

# Verifier 计算 phi(zeta)
phi_v_X = 0 * X
phi_v_zeta = phi_v_X(zeta)
print("phi_v_zeta = ", phi_v_zeta)

L0_v_zeta =  57
z_H_v_zeta =  87
phi_v_zeta =  0


## 验证步

Verifier 验证下面等式是否成立，如果成立则接受，否则拒绝。
$$
\begin{split}
& \bar{q}_L\bar{w}_a+\bar{q}_R\bar{w}_b+ \bar{q}_M\bar{w}_a\bar{w}_b - \bar{q}_O\bar{w}_c+\bar{q}_C + \phi(\zeta)  \\
& \qquad \qquad + \alpha(\bar{z}_{(\omega\cdot\zeta)}\cdot \bar{g}_{(\zeta)}-\bar{z}_{(\zeta)}\cdot \bar{f}_{(\zeta)})+ \alpha^2(L_0(\zeta)\cdot(\bar{z}_{(\zeta)}-1))\overset{?}{=}\bar{t}\cdot z_H(\zeta)
\end{split}
$$

In [88]:
# 计算等式左边
LHS = F_101(q_L_bar * w_a_bar + q_R_bar * w_b_bar + q_M_bar * w_a_bar * w_b_bar - q_O_bar * w_c_bar + q_C_bar + phi_v_zeta + alpha * (z_omega_bar * g_v_bar - z_bar * f_v_bar) + alpha * alpha * (L0_v_zeta * (z_bar - 1)))
print("等式左边计算得到 LHS = ", LHS)

等式左边计算得到 LHS =  81


In [90]:
# 计算等式右边
RHS = F_101(t_bar * z_H_v_zeta)
print("等式右边计算得到 RHS = ", RHS)

等式右边计算得到 RHS =  81


In [92]:
accept = RHS == LHS
print("Verifier 是否通过验证：", accept)

Verifier 是否通过验证： True
